## **GPT2**

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


In [ ]:
# Tải mô hình pre-trained gốc (GPT-2)
original_model_name = "gpt2"
original_model = AutoModelForCausalLM.from_pretrained(original_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Tải mô hình fine-tuned từ Hugging Face
fine_tuned_model_name = "sshreshtaa/gpt2-finetuned"
fine_tuned_model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_name)
# Tạo tokenizer
tokenizer = AutoTokenizer.from_pretrained(original_model_name)

# Xác định hệ số mixing (ví dụ alpha = 0.5)
alpha = 0.5

config.json:   0%|          | 0.00/932 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Fine-mixing: Kết hợp trọng số giữa mô hình gốc và mô hình fine-tuned
for param_orig, param_finetune in zip(original_model.parameters(), fine_tuned_model.parameters()):
    param_orig.data = (1 - alpha) * param_orig.data + alpha * param_finetune.data

# Lưu mô hình đã fine-mix
fine_mix_model_name = "./fine_mix_model"
original_model.save_pretrained(fine_mix_model_name)
tokenizer.save_pretrained(fine_mix_model_name)

('./fine_mix_model/tokenizer_config.json',
 './fine_mix_model/special_tokens_map.json',
 './fine_mix_model/vocab.json',
 './fine_mix_model/merges.txt',
 './fine_mix_model/added_tokens.json',
 './fine_mix_model/tokenizer.json')

In [ ]:
# Đặt eos_token làm pad_token
tokenizer.pad_token = tokenizer.eos_token

def generate_text(model, tokenizer, prompt):
    # Mã hóa chuỗi input và tạo attention mask
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    attention_mask = inputs.attention_mask

    outputs = model.generate(
        inputs.input_ids,
        attention_mask=attention_mask,  # Thêm attention mask
        max_length=100,
        pad_token_id=tokenizer.eos_token_id,  # Sử dụng pad_token_id là eos_token_id
        do_sample =True,
        temperature=0.5,  # Điều chỉnh độ ngẫu nhiên
        top_k=50,         # Giới hạn số lượng từ có xác suất cao nhất
        top_p=0.9,        # Giới hạn xác suất tổng
        repetition_penalty=1.5  # Phạt nếu lặp lại từ
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# Thử sinh văn bản từ mô hình
prompt =  "I am a student and I am 20 years old. I have been studying hard for my exams for the last two months."
generated_text = generate_text(original_model, tokenizer, prompt)
print(f"Generated text with fine-mixng: {generated_text}")


Generated text with fine-mixng: I am a student and I am 20 years old. I have been studying hard for my exams for the last two months. As per the schedule of this school, if you fail your exam then it is not possible to continue with any further studies. However, as long as we are still alive there will be no problem in continuing our education at home or wherever else we can go after graduation from high school. " He also added that he was willing to take his chances on taking an examination again even


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Tải mô hình GPT-2 gốc
model_name = "gpt2"  # Hoặc bạn có thể sử dụng một biến thể khác như "gpt2-medium", "gpt2-large", v.v.
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Định nghĩa prompt
prompt =  "I am a student and I am 20 years old. I have been studying hard for my exams for the last two months. It's been challenging but"

#Tạo đầu vào
inputs = tokenizer(prompt, return_tensors='pt')
attention_mask = inputs['attention_mask']

# Sinh văn bản
outputs = model.generate(
    inputs.input_ids,
    attention_mask=attention_mask,
    max_length=100,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.5
)

# Giải mã và in kết quả
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated text without Fine-Mixing:")
print(generated_text)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text without Fine-Mixing:
I am a student and I am 20 years old. I have been studying hard for my exams for the last two months. It's been challenging but it is good to know that you are ready."
"It was really nice being here," said Sami, who also had his hands up in her pockets before she took off into another room with some snacks on top of them while trying not let their eyes focus down onto each other as they did so often during class sessions this year at UH


## **GPT2-LARGE**

In [ ]:
# Tải mô hình pre-trained gốc (GPT-2)
original_model_name = "gpt2-large"
original_model = AutoModelForCausalLM.from_pretrained(original_model_name)

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Tải mô hình fine-tuned từ Hugging Face
fine_tuned_model_name = "neulab/gpt2-large-finetuned-wikitext103"
fine_tuned_model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_name)
# Tạo tokenizer
tokenizer = AutoTokenizer.from_pretrained(original_model_name)

# Xác định hệ số mixing (ví dụ alpha = 0.5)
alpha = 0.5

config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

In [ ]:
# Fine-mixing: Kết hợp trọng số giữa mô hình gốc và mô hình fine-tuned
for param_orig, param_finetune in zip(original_model.parameters(), fine_tuned_model.parameters()):
    param_orig.data = (1 - alpha) * param_orig.data + alpha * param_finetune.data

# Lưu mô hình đã fine-mix
fine_mix_model_name = "./fine_mix_model_large"
original_model.save_pretrained(fine_mix_model_name)
tokenizer.save_pretrained(fine_mix_model_name)

('./fine_mix_model_large/tokenizer_config.json',
 './fine_mix_model_large/special_tokens_map.json',
 './fine_mix_model_large/vocab.json',
 './fine_mix_model_large/merges.txt',
 './fine_mix_model_large/added_tokens.json',
 './fine_mix_model_large/tokenizer.json')

In [ ]:
# Đặt eos_token làm pad_token
tokenizer.pad_token = tokenizer.eos_token

def generate_text(model, tokenizer, prompt):
    # Mã hóa chuỗi input và tạo attention mask
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    attention_mask = inputs.attention_mask

    outputs = model.generate(
        inputs.input_ids,
        attention_mask=attention_mask,  # Thêm attention mask
        max_length=100,
        pad_token_id=tokenizer.eos_token_id,  # Sử dụng pad_token_id là eos_token_id
        do_sample =True,
        temperature=0.5,  # Điều chỉnh độ ngẫu nhiên
        top_k=50,         # Giới hạn số lượng từ có xác suất cao nhất
        top_p=0.9,        # Giới hạn xác suất tổng
        repetition_penalty=1.5  # Phạt nếu lặp lại từ
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# Thử sinh văn bản từ mô hình
prompt =  "I am a student and I am 20 years old. I have been studying hard for my exams for the last two months."
generated_text = generate_text(original_model, tokenizer, prompt)
print(f"Generated text with fine-mixng: {generated_text}")


Generated text with fine-mixng: I am a student and I am 20 years old. I have been studying hard for my exams for the last two months. My parents are very supportive of me. They don 't want to see any problems in our relationship because they know that it is just an arranged marriage, so we can live together as husband and wife without having issues like this happening again " he said during his interview with CNN Philippines on June 23rd 2015 at Sibutan City Hall ( Malacanang Palace ).
